In [105]:
import pandas as pd
import numpy as np
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import tweepy
import re
from textblob import TextBlob
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [106]:
data = pd.read_excel('Hasil_pelabelan_manual(2).xlsx')
data.head()

,Tweet_bersih,Klasifikasi
0,Eid holiday homecoming Lebaran public transpor...,Positif
1,"Last year's Eid, three were quiet, Pol. It fel...",Negatif
2,"Eid homecoming year, you can go home and you c...",Netral
3,"akiminow Eid special year going home to teak, ...",Positif
4,"cross-flow engineering for Lebaran, the value ...",Positif


In [107]:
data_tweet = list(data['Tweet_bersih'])
polaritas = 0

status = []
total_positif = total_negatif = total_netral = total = 0

for i, tweet in enumerate(data_tweet):
  analysis = TextBlob(tweet)
  polaritas += analysis.polarity

  if analysis.sentiment.polarity > 0.0:
    total_positif += 1
    status.append('Positif')
  elif analysis.sentiment.polarity == 0.0:
    total_netral += 1
    status.append('Netral')
  else :
    total_negatif += 1
    status.append('Negatif')

  total += 1

print(f'Hasil Analisis Data :\n Positif = {total_positif}\nNetral = {total_netral}\nNegatif = {total_negatif} ' )
print(f'\nTotal Data : {total}')

Hasil Analisis Data :
 Positif = 39
Netral = 38
Negatif = 8 

Total Data : 85


In [108]:
#memisahkan data dengan kelas
from sklearn.model_selection import train_test_split

x = data['Tweet_bersih'].values
y = data['Klasifikasi'].values

#vektorisasi
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf = TfidfVectorizer(ngram_range=(1,3))
x = tf_idf.fit_transform(x)

In [109]:
#smote
from imblearn.over_sampling import SMOTE
from collections import Counter

sm = SMOTE()
X_smote, y_smote = sm.fit_resample(x, y)
print(sorted(Counter(y_smote).items()))

[('Negatif', 39), ('Netral', 39), ('Positif', 39)]


In [110]:
#classifier data
nb = MultinomialNB()
nb.fit(X_smote, y_smote)
preds = nb.predict(X_smote)

In [111]:
print(classification_report(y_smote, preds))

              precision    recall  f1-score   support

     Negatif       1.00      1.00      1.00        39
      Netral       1.00      1.00      1.00        39
     Positif       1.00      1.00      1.00        39

    accuracy                           1.00       117
   macro avg       1.00      1.00      1.00       117
weighted avg       1.00      1.00      1.00       117



In [112]:
print(accuracy_score(y_smote, preds))

1.0
